<a href="https://colab.research.google.com/github/adi-dev-vikram/StreamLitApps/blob/main/NFL_QB_Stats_Explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit

     |████████████████████████████████| 9.2 MB 8.3 MB/s 
     |████████████████████████████████| 76 kB 4.2 MB/s 
     |████████████████████████████████| 164 kB 57.2 MB/s 
     |████████████████████████████████| 180 kB 45.9 MB/s 
     |████████████████████████████████| 111 kB 52.0 MB/s 
     |████████████████████████████████| 4.3 MB 47.2 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 126 kB 55.9 MB/s 
     |████████████████████████████████| 791 kB 50.3 MB/s 
     |████████████████████████████████| 374 kB 56.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.24 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.6.0 which is incompatible.
google-colab 1.0.0 requires ipytho

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import base64
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#qbs now adding more tds add this later

st.title('NFL Overall QB Stats Explorer')

st.markdown("""
This app performs simple webscraping of NFL player stats focused on QBs wrt to their teams performance!
* **Data source:** [pro-football-reference.com](https://www.pro-football-reference.com/).
""")

# for eg https://www.pro-football-reference.com/players/R/RodgAa00.htm created from name: Aaron Rodgers
def create_search_query(name):
  name_list = name.split()
  first_str = name_list[1][0:4]
  second_str = name_list[0][0:2]
  search_str = "https://www.pro-football-reference.com/players/" + str(list(name_list[1])[0]) + "/" + first_str + second_str + "00.htm"
  df_list = pd.read_html(search_str)[1]
  st.title("This guys stats")  # add a title
  st.write(df_list)



st.sidebar.header('User Input Features')
selected_year = st.sidebar.selectbox('Year', list(reversed(range(2010,2021))))
qb_name = st.text_input("Enter the QB's name : ", "Type Here ...")

if(st.button('Submit')):
    result = qb_name.title()
    create_search_query(result)
    st.success(result)


@st.cache
def load_data(year):
    url = "https://www.pro-football-reference.com/years/" + str(year) + "/passing.htm"
    df_list = pd.read_html(url)[0]
    df_list=df_list.drop('Rk', 1)
    playerstats = df_list
    return playerstats
playerstats = load_data(selected_year)

# Sidebar - Team selection
sorted_unique_team = sorted(playerstats.Tm.unique())
selected_team = st.sidebar.multiselect('Team', sorted_unique_team, sorted_unique_team)

# Sidebar - Position selection
unique_pos = ['RB','QB','WR','FB','TE']
selected_pos = st.sidebar.multiselect('Position', unique_pos, unique_pos)

# Filtering data
df_selected_team = playerstats[(playerstats.Tm.isin(selected_team)) & (playerstats.Pos.isin(selected_pos))]

st.header('Display Player Stats of Selected Team(s)')
st.write('Data Dimension: ' + str(df_selected_team.shape[0]) + ' rows and ' + str(df_selected_team.shape[1]) + ' columns.')
st.dataframe(df_selected_team)

# Download NBA player stats data
# https://discuss.streamlit.io/t/how-to-download-file-in-streamlit/1806
def filedownload(df):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()  # strings <-> bytes conversions
    href = f'<a href="data:file/csv;base64,{b64}" download="playerstats.csv">Download CSV File</a>'
    return href

st.markdown(filedownload(df_selected_team), unsafe_allow_html=True)

# Heatmap
if st.button('Intercorrelation Heatmap'):
    st.header('Intercorrelation Matrix Heatmap')
    df_selected_team.to_csv('output.csv',index=False)
    df = pd.read_csv('output.csv')

    corr = df.corr()
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    with sns.axes_style("white"):
        f, ax = plt.subplots(figsize=(7, 5))
        ax = sns.heatmap(corr, mask=mask, vmax=1, square=True)
    st.pyplot()

Overwriting app.py


In [ ]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 5.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=f3ac02084d10b0ffbc03f78af1055116004d563a4997623053bacc5ce2817ee1
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
from pyngrok import ngrok

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-12-17 05:33:11--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  3.33MB/s    in 5.0s    

2021-12-17 05:33:16 (2.66 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [ ]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: http://a228-34-125-39-45.ngrok.io


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

npx: installed 22 in 2.839s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.227.152.65:8501

your url is: https://shaggy-goat-22.loca.lt
2021-12-27 10:23:25.248 NumExpr defaulting to 2 threads.


# New Section